In [1]:
#######
## 
## testing the functions zach wrote for data download
## doesn't use wget and fully "pythonized" - uses requests instead
## using asf api to generate URLS


# import libraries
import numpy as np
import re
import zipfile
import getpass
from osgeo import gdal 
import os  # for chdir, getcwd, path.basename, path.exists
import pandas as pd # for DatetimeIndex
import codecs # for text parsing code
import netrc
import rasterio as rio
from rasterio.plot import show # plotting raster data
from rasterio.plot import show_hist #histograms of raster data
import glob
import requests
from shapely.geometry import Polygon, mapping
from datetime import datetime
from subprocess import PIPE, Popen
from os.path import join, isdir, isfile, basename
import progressbar
from tqdm import tqdm
import logging

In [2]:
# input NASA Earthdata credentials here
ASF_USER = input("Enter Username: ")
ASF_PASS = getpass.getpass("Enter Password: ")

Enter Username:  jacktarricone
Enter Password:  ············


In [3]:
log = logging.getLogger(__name__)

In [4]:
print(log)

<Logger __main__ (WARNING)>


### Function definitions to query ASF APi

In [5]:
def build_bbox_string(polygon):
    '''
    Builds the string to include in the ASF search request. The bbox consists of 4 comma-separated numbers: lower left longitude,latitude, and upper right longitude,latitude.
    
    Parmeters
    ----------
    polygon: shapely polygon
    
    Returns
    ----------
    polygon_string : string
        String to include in the ASF request
    '''
    points = mapping(polygon)['coordinates'][0]
    lower_left = points[0]
    upper_right = points[2]
    bbox_string = f'{lower_left[0]},{lower_left[1]},{upper_right[0]},{upper_right[1]}'
        
    return bbox_string

In [6]:
def search_asf(platform, processingLevel, start, end, polygon, output_format):
    '''
    Search the ASF platform for images given the input parameters
    
    Parameters
    ----------
        platform : string
            Name of the imaging platform. Defaults to UAVSAR, but a list of supported platform is available on the ASF website
        processingLevel : string
            Processing level of the imaging product. 
            Possible values for UAVSAR : (KMZ, PROJECTED, PAULI, PROJECTED_ML5X5, STOKES, AMPLITUDE, BROWSE, COMPLEX, DEM_TIFF, PROJECTED_ML3X3, METADATA, AMPLITUDE_GRD, INTERFEROMETRY, INTERFEROMETRY_GRD, THUMBNAIL)
        start : datetime object
            Start date of the search period.
        end : datetime object
            End date of the search period.
        polygon : shapely polygon defining the Area of Interest,
        output_format: string
            Format being returned by the ASF API. Values : CSV, JSON, KML, METALINK, COUNT, DOWNLOAD, GEOJSON
        
    Returns
    -------
    Ouputs a search file
    '''
    base = 'https://api.daac.asf.alaska.edu/services/search/param'
    start_date = start.strftime('%Y-%m-%dT%H:%M:%SUTC')
    end_date = end.strftime('%Y-%m-%dT%H:%M:%SUTC')
    aoi_string = build_bbox_string(polygon)
    payload = {
        'platform': platform,
        'processingLevel': processingLevel,
        'start': start,
        'end': end,
        'bbox': aoi_string,
        'output': output_format
    }
    r = requests.get(base, params=payload)
    
    return r.json()

In [7]:
def stream_download(url, output_f):
    """
    Args:
        url: url to download
        output_f: path to save the data to
    """
    print(f"\n\nDownloading {url}\nSaving to {output_f}")
    r = requests.get(url, stream=True)

    if r.status_code == 200:
        #bar = progressbar.ProgressBar(max_value=progressbar.UnknownLength)
        with open(output_f, 'wb') as f:
            for i, chunk in tqdm(enumerate(r), description = f'Downloading {basename(url)}'):
                f.write(chunk)
                #bar.update(i)
    else:
        print(f"HTTP CODE {r.status_code}. Skipping download!")

In [8]:
def download_InSAR(url, output_dir, polarizations=['HH', 'HV', 'VH', 'VV'],
                    file_types=['amp1', 'amp2', 'cor', 'int']):
    """
    Downloads uavsar InSAR files from a url.
    Args:
        url (string): A url containing uavsar flight data. Can be from JPL or ASF
        output_dir (string): Directory to save the data in
    Returns:
        None
    Raises:
        ValueError: If the base flight name is missing the polarization HH
    """
    log.info('Starting download of {url}...')

    local = join(output_dir, basename(url))
    # Checks for existence of url
    try:
        r = requests.get(url)
    except requests.HTTPError as e:
        log.warning(f'{url} returned {e}')

    # Make the output dir if it doesn't exist
    if not isdir(output_dir):
        os.makedirs(output_dir)

    if not isfile(local):
        stream_download(url, local)
    else:
        log.info(f"{local} already exists, skipping download!")

    # Download the ann file always.
    ext = url.split('.')[-2]
    ann_url = url.replace(f"{ext}.grd", "ann")
    ann_local = local.replace(f"{ext}.grd", "ann")

    if not isfile(ann_local):
        stream_download(ann_url, ann_local)
    else:
        log.info(f"{ann_local} already exists, skipping download!")

In [9]:
# define search parameters for sierra flight line
sierra_polygon = Polygon([(-119.9697,37.4631),(-118.9576,37.4631),(-118.9576,38.7211),(-119.969,38211)])
start_date = datetime.strptime('2020-02-28 11:00:00', '%Y-%m-%d %H:%M:%S') 
end_date = datetime.strptime('2020-03-11 11:00:00', '%Y-%m-%d %H:%M:%S') 

In [10]:
# query API to generate one over pass
results = search_asf(platform='UAVSAR', processingLevel='INTERFEROMETRY_GRD', 
                    start=start_date, end=end_date, polygon=sierra_polygon, output_format='JSON')[0]

# print number of products
print(f'{len(results)} product(s) found')

1 product(s) found


In [11]:
# define url
for i in results:
        downloadUrl = i['downloadUrl']
        print(downloadUrl)

https://datapool.asf.alaska.edu/INTERFEROMETRY_GRD/UA/sierra_17305_20014-000_20016-005_0014d_s01_L090_01_int_grd.zip


In [ ]:
download_InSAR(url = downloadUrl, 
               output_dir = '/Users/jacktarricone/Desktop/zach_test/',
               polarizations = ['HH'],
               file_types=['amp1', 'amp2', 'cor', 'int'])

In [12]:
downloadUrl

'https://datapool.asf.alaska.edu/INTERFEROMETRY_GRD/UA/sierra_17305_20014-000_20016-005_0014d_s01_L090_01_int_grd.zip'

In [ ]:
# get new path for folder of insar data just downloaded
new_path_list = glob.glob('/Users/jacktarricone/ch2_sierra_data/sierra/*')
new_path = new_path_list[3] # select first list elemet
print(new_path)

In [ ]:
# convert files to geotiffs and save in same folder
uavsar_tiff_convert(new_path)

In [ ]:
# remove unwanted files
os.chdir(new_path)
grd = glob.glob('*.grd') #define .grd
hdr = glob.glob('*.hdr*') #define .hdr
int_file = glob.glob('*.int*') #define .int

# remove both
for f in grd:
    os.remove(f)
    
for f in hdr:
    os.remove(f)
    
for f in int_file:
    os.remove(f)

## Define only the HH files

In [ ]:
# coherence
for cor in glob.glob("*HH*cor.grd.tif"):
    print(cor)

# unwrapped phase
for unw in glob.glob("*HH*unw.grd.tif"):
    print(unw)

# dem used in processing
for dem in glob.glob("*HH*hgt.grd.tif"):
    print(dem)

In [ ]:
# open using rio
cor_rast = rio.open(cor)
unw_rast = rio.open(unw)
dem_rast = rio.open(dem)

In [ ]:
# test plot
show(cor_rast)
show(unw_rast)
show(dem_rast)

In [ ]:
# unw
with rio.open(unw) as unw_raw:
    unw_array = unw_raw.read(1)

# convert all 0's to nan
unw_array[unw_array==0] = np.nan

#cor
with rio.open(cor) as cor_raw:
    cor_array = cor_raw.read(1)

cor_array[cor_array==0] = np.nan

#dem, no data value is -10000
with rio.open(dem) as dem_raw:
    dem_array = dem_raw.read(1)

dem_array[dem_array == -10000] = np.nan

In [ ]:
# plot NaN corrected data
show(unw_array)
show(cor_array)
show(dem_array)